# Quickstart

## **Cơ chế hoạt động**

**Tinh chỉnh (Fine-tuning)** một mô hình ngôn ngữ thông qua **PPO (Proximal Policy Optimization - Tối ưu hóa Chính sách Gần đúng)** bao gồm ba bước chính:

### **1. Rollout (Triển khai)**
Mô hình ngôn ngữ tạo ra một phản hồi hoặc phần tiếp theo dựa trên một **`query`** (truy vấn), ví dụ như phần đầu của một câu. Về cơ bản, đây là lúc mô hình "hành động" để tạo ra văn bản.

### **2. Evaluation (Đánh giá)**
`Query` và phản hồi được đánh giá bằng một hàm, một mô hình khác, phản hồi từ con người, hoặc sự kết hợp của chúng. Điều quan trọng là quá trình này phải đưa ra một **giá trị vô hướng (scalar value)** cho mỗi cặp `query`/phản hồi. Quá trình tối ưu hóa sẽ nhắm đến việc **tối đa hóa** giá trị này. Giá trị này hoạt động như một "phần thưởng" (reward). 🏆

### **3. Optimization (Tối ưu hóa)**
Đây là phần phức tạp nhất. Trong bước tối ưu hóa, các cặp `query`/phản hồi được sử dụng để tính toán `log-probabilities` (log xác suất) của các `tokens` trong chuỗi. Việc này được thực hiện với mô hình đang được huấn luyện và một **mô hình tham chiếu (reference model)**, thường là mô hình đã được huấn luyện trước đó (pre-trained model).

**Phân kỳ KL (KL-divergence)** giữa đầu ra của hai mô hình này được sử dụng như một tín hiệu phần thưởng bổ sung. Mục đích là để đảm bảo các phản hồi được tạo ra không đi quá xa so với ngôn ngữ của mô hình tham chiếu. Cuối cùng, mô hình ngôn ngữ đang hoạt động (active language model) được huấn luyện bằng thuật toán **PPO**.

Toàn bộ quy trình được minh họa trong hình sau:

![](images/trl_overview.png)

## Minimal example

In [1]:
import torch
from transformers import GPT2Tokenizer
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer 

# 1. Load pretrained model
model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

/home/dino/Documents/learn-llm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{"timestamp":"2025-07-22T07:57:57.286955Z","level":"WARN","fields":{"message":"Reqwest(reqwest::Error { kind: Request, source: hyper_util::client::legacy::Error(Connect, Custom { kind: Other, error: Os { code: 110, kind: TimedOut, message: \"Connection timed out\" } }) }). Retrying..."},"filename":"/home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":242}
{"timestamp":"2025-07-22T07:57:57.286987Z","level":"WARN","fields":{"message":"Reqwest(reqwest::Error { kind: Request, source: hyper_util::client::legacy::Error(Connect, Custom { kind: Other, error: Os { code: 110, kind: TimedOut, message: \"Connection timed out\" } }) }). Retrying..."},"filename":"/home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":242}
{"timestamp":"2025-07-22T07:57:57.287039Z","level":"WARN","fields":{"message":"Retry attempt #0. Sleeping 804.551536ms before the next attempt"},"filename":"/root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/reqwest-retr

Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

In [ ]:
tokenizer.eos_token

In [ ]:
# 2. initialize trainer 
ppo_config = {
    "mini_batch_size": 1,
    "batch_size": 1
}
config = PPOConfig(**ppo_config)
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer)

In [ ]:
# 3. encode a query
query_txt = "This morning I went to the "
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(model.pretrained_model.device)

In [ ]:
# 4. generate model response
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 20,
}
response_tensor = ppo_trainer.generate([item for item in query_tensor], return_prompt=False, **generation_kwargs)
response_txt = tokenizer.decode(response_tensor[0])
response_txt

In [ ]:
# 5. define a reward for response
# (this could be any reward such as human feedback or output from another model)
reward = [torch.tensor(1.0, device=model.pretrained_model.device)]
reward

In [ ]:
# 6. train model with ppo
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)
# In general, you would run steps 3-6 in a for-loop and run it on many diverse queries. 
# You can find more realistic examples in the examples section.

## How to use a trained model

In [ ]:
#save model
model.save_pretrained("trained_model/quickstart_ppo")

In [ ]:
# load model
from transformers import AutoModelForCasualLM
loaded_model = AutoModelForCasualLM.from_pretrained("trained_model/quickstart_ppo")

In [ ]:
# You can also load your model with AutoModelForCausalLMWithValueHead 
# if you want to use the value head, for example to continue training.

from trl.model import AutoModelForCasualLMWithValueHead

model = AutoModelForCasualLMWithValueHead.from_pretrained("trained_model/quickstart_ppo")